# Practical Uses of SQL For Making Business Decisions

The goal for this project is to come up with business decisions based on SQL queries. This project is based off the Chinook Database, with the following schema.

<img src = 'chinook-schema.svg' width=500>

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql
SELECT name, type FROM sqlite_master
WHERE type IN ('table', 'view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The first thing I am going to want to do is get a look at what tables and views the database has. In addition to all the tables in the schema, there are two views:

1. customer_usa: Customers in the USA
2. customer_gt_90_dollars: Customers who have purchased more than 90 dollars in music

A new record label has signed on to distribute through our store. We've come to an agreement where I will sell three of the 4 new albums they are selling:

|Artist Name| 	Genre|
|---|---|
|Regal |	Hip-Hop|
|Red Tone |	Punk|
|Meteor and the Girls |	Pop
|Slim Jim Bites |	Blues

Since I don't know anything about the individual artists, the only information I have to know what I want to purchase is the name of the artist and the genre.

In general, unless it's a big named artist, the name has low impact on the sales. These names are fairly neutral in sentiment, so I will rely on the genre to be the deciding factor of my choice.

In [34]:
%%sql
DROP VIEW genre_count;
CREATE VIEW genre_count AS
SELECT g.name genre_name,
       SUM(il.quantity) number_of_tracks
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC;

SELECT genre_name,
       number_of_tracks,
      ROUND(CAST(number_of_tracks AS FLOAT) * 100 / (SELECT SUM(number_of_tracks)
                                          FROM genre_count),2) genre_percent
FROM genre_count;

 * sqlite:///chinook.db
Done.
Done.
Done.


genre_name,number_of_tracks,genre_percent
Rock,2635,55.39
Metal,619,13.01
Alternative & Punk,492,10.34
Latin,167,3.51
R&B/Soul,159,3.34
Blues,124,2.61
Jazz,121,2.54
Alternative,117,2.46
Easy Listening,74,1.56
Pop,63,1.32


Since the 4 options were Punk, Pop, Hip Hop, and Blues, and my sales quantities are shown above. It looks like I will be ordering the Punk, Blues, and Pop albums and leaving off the Hip Hop album. As it stands, since Pop and Hip Hop are fairly close to each other around 1\% of sales, I would probably just take the punk and blues albums.

Something that I note from my own music experience, Alternative \& punk are so different that they should not be combined into one genre. Punk by itself is already a diverse genre, and Alternative is rather all encompassing. These two being grouped together may be throwing the representation of the sales entirely out of whack

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1
